In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scvi
import scanpy as sc

Global seed set to 0
/restricted/projectnb/camplab/home/yin/.conda/envs/totalvi/lib/python3.9/site-packages/chex/_src/pytypes.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


In [2]:
# Import data

# datadir_name = 'data_pbmc5knextgem'
# datadir_name = 'data_pbmc10khealthdonor'
# datadir_name = 'data_malt10k'

# datadir = '/rprojectnb2/camplab/home/yin/poisson/' + datadir_name


# Import data
datadir_name = 'young_aged'
datadir = '/rprojectnb2/camplab/home/yin/decontX_ADT/zhangetal/young-aged'


adata_filtered = sc.read_10x_mtx(
    datadir + '/filtered_feature_bc_matrix', 
    var_names='gene_symbols',
    gex_only= False)


'''
# Import raw count matrix
adata_raw = sc.read_10x_mtx(
    datadir + '/raw_feature_bc_matrix',
    var_names='gene_symbols',
    gex_only= False)
'''




"\n# Import raw count matrix\nadata_raw = sc.read_10x_mtx(\n    datadir + '/raw_feature_bc_matrix',\n    var_names='gene_symbols',\n    gex_only= False)\n"

In [3]:
print(adata_filtered)

AnnData object with n_obs × n_vars = 30569 × 31111
    var: 'gene_ids', 'feature_types'


In [4]:
adata_filtered.layers['counts'] = adata_filtered.X.copy()

In [5]:
adata_filtered.var['feature_types']

Xkr4        Gene Expression
Gm1992      Gene Expression
Gm37381     Gene Expression
Rp1         Gene Expression
Sox17       Gene Expression
                 ...       
H-HTO-2    Antibody Capture
H-HTO-3    Antibody Capture
H-HTO-4    Antibody Capture
H-HTO-5    Antibody Capture
H-HTO-6    Antibody Capture
Name: feature_types, Length: 31111, dtype: object

In [6]:
# Split data into protein and rna

protein = adata_filtered[:, adata_filtered.var['feature_types'] != 'Gene Expression']
print(protein)
rna = adata_filtered[:, adata_filtered.var['feature_types'] == 'Gene Expression']
print(rna)

View of AnnData object with n_obs × n_vars = 30569 × 49
    var: 'gene_ids', 'feature_types'
    layers: 'counts'
View of AnnData object with n_obs × n_vars = 30569 × 31062
    var: 'gene_ids', 'feature_types'
    layers: 'counts'


In [7]:
# Remove HTO tag

hto_f = ['HTO' in name for name in protein.var_names]
hto_f = np.array(hto_f)
protein = protein[:,~hto_f]

print(protein)

hto_f = ['ADT' in name for name in protein.var_names]
hto_f = np.array(hto_f)
protein = protein[:,~hto_f]

print(protein)

adt_f = np.sum(protein.X > 0, 0) > 50
protein = protein[:,adt_f]

print(protein)

View of AnnData object with n_obs × n_vars = 30569 × 35
    var: 'gene_ids', 'feature_types'
    layers: 'counts'
View of AnnData object with n_obs × n_vars = 30569 × 35
    var: 'gene_ids', 'feature_types'
    layers: 'counts'
View of AnnData object with n_obs × n_vars = 30569 × 31
    var: 'gene_ids', 'feature_types'
    layers: 'counts'


In [8]:
# QC

# calc qc metrics
rna.var['mito'] = rna.var_names.str.startswith("MT-")
print('Num of MT genes: ' + str(sum(rna.var['mito'])))

if sum(rna.var['mito']) == 0:
    rna.var['mito'] = rna.var_names.str.startswith("mt-")
    print('Num of mt genes: ' + str(sum(rna.var['mito'])))

sc.pp.calculate_qc_metrics(rna,
                           qc_vars=["mito"],
                           var_type='genes',
                           percent_top = None,
                           inplace=True)

sc.pp.calculate_qc_metrics(protein,
                           var_type='adt',
                           percent_top = None,
                           inplace=True)




/scratch/136144.1.ood/ipykernel_5995/7817696.py:4: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  rna.var['mito'] = rna.var_names.str.startswith("MT-")


Num of MT genes: 0
Num of mt genes: 13


/restricted/projectnb/camplab/home/yin/.conda/envs/totalvi/lib/python3.9/site-packages/scanpy/preprocessing/_qc.py:135: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[obs_metrics.columns] = obs_metrics


In [9]:
# Remove high mitocondrial percentage cells
# Remove 0 count cells
# Remove outliers

flt = np.logical_and.reduce((rna.obs['pct_counts_mito'] < 14,
                            rna.obs['total_counts'] > 0,
                            protein.obs['total_counts'] > 0,
                            rna.obs['total_counts'] > np.quantile(rna.obs['total_counts'], 0.01),
                            rna.obs['total_counts'] < np.quantile(rna.obs['total_counts'], 0.99),
                            protein.obs['total_counts'] > np.quantile(protein.obs['total_counts'], 0.01),
                            protein.obs['total_counts'] < np.quantile(protein.obs['total_counts'], 0.99)))
 
    
    

print('Num of cells after filtering: ' + str(sum(flt)))

print(sum(rna.obs['pct_counts_mito'] < 14))
print(sum(rna.obs['total_counts'] > 0))
print(sum(protein.obs['total_counts'] > 0))
print(np.quantile(rna.obs['total_counts'], 0.01))


Num of cells after filtering: 27712
28784
30569
30100
483.68


In [10]:
# Apply filter

feature_flt = [feature in rna.var_names or feature in protein.var_names for feature in adata_filtered.var_names]

adata_filtered_clean = adata_filtered[flt,feature_flt]
print(adata_filtered_clean.var_names)


Index(['Xkr4', 'Gm1992', 'Gm37381', 'Rp1', 'Sox17', 'Gm37323', 'Mrpl15',
       'Lypla1', 'Gm37988', 'Tcea1',
       ...
       'CITE_PD-1', 'CITE_PD-L1', 'CITE_CD169-Siglec-1', 'CITE_Siglec-H',
       'CITE_TMEM119', 'CITE_XCR1', 'CITE_CD24', 'CITE_CD103', 'CITE_CD64',
       'CITE_CD83'],
      dtype='object', length=31093)


In [11]:
# Organize: https://docs.scvi-tools.org/en/stable/tutorials/notebooks/data_loading.html

scvi.data.organize_cite_seq_10x(adata_filtered_clean)
print(adata_filtered_clean)
print(adata_filtered_clean.obsm['protein_expression'].shape)


AnnData object with n_obs × n_vars = 27712 × 31062
    var: 'gene_ids', 'feature_types'
    obsm: 'protein_expression'
    layers: 'counts'
(27712, 31)


In [12]:
# Setup for totalVI: https://docs.scvi-tools.org/en/stable/tutorials/notebooks/totalVI.html

sc.pp.normalize_total(adata_filtered_clean, target_sum=1e4)
sc.pp.log1p(adata_filtered_clean)

sc.pp.highly_variable_genes(
    adata_filtered_clean,
    n_top_genes=4000,
    flavor="seurat_v3",
    subset=True,
    layer="counts"
)

print(adata_filtered_clean)

AnnData object with n_obs × n_vars = 27712 × 4000
    var: 'gene_ids', 'feature_types', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'log1p', 'hvg'
    obsm: 'protein_expression'
    layers: 'counts'


In [13]:
# Run totalVI
scvi.model.TOTALVI.setup_anndata(
    adata_filtered_clean,
    protein_expression_obsm_key="protein_expression",
    layer="counts"
)

vae = scvi.model.TOTALVI(adata_filtered_clean, latent_distribution="normal")
vae.train()

INFO     Using column names from columns of adata.obsm['protein_expression']                 
INFO     Computing empirical prior initialization for protein background.                    


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 400/400: 100%|██████████| 400/400 [2:40:22<00:00, 24.06s/it, loss=1.02e+03, v_num=1]


In [14]:
# Get results from trained model
adata_filtered_clean.obsm["X_totalVI"] = vae.get_latent_representation()

adata_filtered_clean.layers["denoised_rna"], adata_filtered_clean.obsm["denoised_protein"] = vae.get_normalized_expression(
    n_samples=25,
    return_mean=True
)


In [15]:
print(adata_filtered_clean)
print(adata_filtered_clean.obsm["denoised_protein"])

AnnData object with n_obs × n_vars = 27712 × 4000
    obs: '_scvi_labels', '_scvi_batch'
    var: 'gene_ids', 'feature_types', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'log1p', 'hvg', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'protein_expression', 'X_totalVI', 'denoised_protein'
    layers: 'counts', 'denoised_rna'
                    CITE_CCR2-CD192  CITE_CD117-c-kit  CITE_CD11b  CITE_CD11c  \
AAACCCAAGCCGTTAT-1         0.234600          6.299818  140.652878   22.322241   
AAACCCAAGGACGCTA-1         0.209244          0.005719  226.183640    0.616301   
AAACCCACACAACCGC-1         0.232468          0.165325   14.420986    1.996154   
AAACCCATCGCAACAT-1         0.207793          0.004741  215.626129    0.677236   
AAACGAAAGTAAGCAT-1         0.280955         70.391823   24.819355    1.902060   
...                             ...               ...         ...         ...   
TTTGTTGGTTGACGGA-2         0.170946          0.008394  151.7364

In [16]:
# Save reults to plot in R
adata_filtered_clean.obsm["denoised_protein"].to_csv('./totalvi_denoised_' + datadir_name + '.csv')